In [1]:
import cv2 as cv
import numpy as np
from time import sleep
import os
os.chdir('../../data')

In [2]:
y1 = 600
offset = 5
width = 50
height = 50
detect = []
cars = 0
capWidth = 1080
capHeight = 720
def get_center(x, y, w, h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx, cy

In [ ]:
cap = cv.VideoCapture('road2.mp4')
BGS = cv.createBackgroundSubtractorMOG2(history=40, varThreshold=40, detectShadows=False)
forcc = cv.VideoWriter_fourcc(*'mp4v')
writer = cv.VideoWriter('cars.mp4', forcc, 30, (capWidth, capHeight))

while cap.isOpened():
    rate, frame = cap.read()
    if rate:
        frame = cv.resize(frame, (capWidth, capHeight))
        grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        blur = cv.GaussianBlur(grayFrame, (3, 3), 5)
        img_sub = BGS.apply(blur)
        dilat = cv.dilate(img_sub, np.ones((7, 7)))

        contur, h = cv.findContours(dilat, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        cv.line(frame, (20, y1), (1050, y1), (177, 130, 40), 3)

        for i, c in enumerate(contur):
            x, y, w, h = cv.boundingRect(c)

            if w >= width and h>= height:
                cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                center = get_center(x, y, w, h)
                detect.append(center)
                cv.circle(frame, center, 4, (0, 0, 255), -1)

                for x, y in detect:
                    if y < (y1+offset) and y > (y1-offset):
                        cars+=1
                        cv.line(frame, (20, y1), (1050, y1), (0, 127, 255), 3)
                        detect.remove((x, y))
                        # print('Number of cars detected : '+str(cars))

        cv.putText(frame, 'NUMBER OF VEHICLE : '+str(cars), (150, 70), cv.FONT_HERSHEY_SIMPLEX,2 , (255, 0, 255), 3)
        writer.write(frame)
        # cv.imshow('imag_sub', dilat)
        cv.imshow('frame', frame)
        k = cv.waitKey(40)
        if k == 27:
            break
    else:
        break
writer.release()
cap.release()
cv.destroyAllWindows()